## Effect of changing the number of 'reps'

### Aim: To investigate the situation where we increase the number of 'reps' for different alpha values.

'reps' refers to the number of times we repeat the ansatz. In Qiskit 'reps' is set to 1 by default.In this notebook, we will work with the 10 asset scenario using the qasm_simulator. 

Here we increase reps to 2.



### Expected Result:

Better approximation of optimal functional value.

In [1]:
#importing all the necessary packages
from qiskit.algorithms.optimizers import COBYLA,CG,POWELL, GradientDescent
from qiskit.algorithms import NumPyMinimumEigensolver, VQE,QAOA
from qiskit.opflow import PauliExpectation, CVaRExpectation
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import LinearEqualityToPenalty
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp
from qiskit import execute, Aer
from qiskit.utils import algorithm_globals

import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model

In [2]:
#maruti,ongc,tatasteel,hindalco,icici,britannia,ultra,wipro,apollo,jublfoods
mu10 = np.array([-0.01278393459,-0.01757157357,0.00153724791,-0.015034464,-0.015034464,-0.01387133952,-0.01447346195,
                -0.01674339724,-0.0132387857,-0.01195091376])

sigma10 = np.array(
    [
        [0.0157149876, 0.01543639416,0.01557457106, 0.01548754824, 0.01551981118,0.01552742087,0.01551068247,0.01549833146,0.01555422578,0.01560541445],
        [0.01543639416,0.01626383626, 0.01518401219, 0.01572622731, 0.01547586679, 0.01523653022,0.01540274745,0.01539792764,0.01529484431,0.01522477939],
        [0.01557457106, 0.01518401219, 0.03221621237,0.01521450154, 0.01560453568, 0.01568907088,0.01536051938,0.01517390478,0.01544208176,0.01533647982],
        [0.01548754824, 0.01572622731,0.01521450154, 0.01609035237, 0.01563323403, 0.01546962982,0.01556278296,0.01563908817,0.01556359729,0.01560570359],
        [0.01551981118, 0.01547586679,0.01560453568, 0.01563323403, 0.01566618101, 0.01551780546,0.01554861694,0.01554772412,0.01554744909,0.01561664097],
        [0.01552742087,0.01523653022, 0.01568907088, 0.01546962982, 0.01551780546, 0.0156690731,0.01550048831,0.01547100286,0.01552910437,0.0155765202],
        [0.01551068247,0.01540274745,0.01536051938,0.01556278296,0.01554861694,0.01550048831,0.01570878379,0.01551585945,0.01551281895,0.01563468601],
        [0.01549833146,0.01539792764,0.01517390478,0.01563908817,0.01554772412,0.01547100286,0.01551585945,0.01568793576,0.0155514752,0.01569287103],
        [0.01555422578,0.01529484431,0.01544208176,0.01556359729,0.01554744909,0.01552910437,0.01551281895,0.0155514752,0.0157783728,0.01565209048],
        [0.01560541445,0.01522477939,0.01533647982,0.01560570359,0.01561664097,0.0155765202,0.01563468601,0.01569287103,0.01565209048,0.0161425637]
    ]
)

In [3]:
# prepare problem instance
n = 10  # number of assets
q = 0.5  # risk factor
budget = n // 2  # budget
penalty = 2 * n  # scaling of penalty term

In [4]:
#Using cobyla optimizer and qasm simulator. We are using CVaR expectation and hence, created a list of alpha values

# set classical optimizer
maxiter = 100
optimizer = COBYLA(maxiter=maxiter)

# set variational ansatz
#ansatz = RealAmplitudes(n, reps=2)
#m = ansatz.num_parameters

# set backend
backend_name = "qasm_simulator"  # use this for QASM simulator
# backend_name = 'aer_simulator_statevector'  # use this for statevector simlator
backend = Aer.get_backend(backend_name)

# run variational optimization for different values of alpha
alphas = [1.0, 0.50, 0.25,0.30,0.40,0.45]  # confidence levels to be evaluated

In [6]:
# create docplex model
mdl = Model("portfolio_optimization")
x = mdl.binary_var_list(range(n), name="x")
objective = mdl.sum([mu10[i] * x[i] for i in range(n)])
objective -= q * mdl.sum([sigma10[i, j] * x[i] * x[j] for i in range(n) for j in range(n)])
mdl.maximize(objective)
mdl.add_constraint(mdl.sum(x[i] for i in range(n)) == budget)

# case to
qp = from_docplex_mp(mdl)

In [7]:
linear2penalty = LinearEqualityToPenalty(penalty=penalty)
qp = linear2penalty.convert(qp)
operator , offset = qp.to_ising()

In [8]:
%%time
# dictionaries to store optimization progress and results
objectives = {alpha: [] for alpha in alphas}  # set of tested objective functions w.r.t. alpha
results = {}  # results of minimum eigensolver w.r.t alpha

# callback to store intermediate results
def callback(i, params, obj, stddev, alpha):
    # we translate the objective from the internal Ising representation
    # to the original optimization problem
    objectives[alpha] += [-(obj + offset)]


# loop over all given alpha values
for alpha in alphas:

    # initialize CVaR_alpha objective
    cvar_exp = CVaRExpectation(alpha, PauliExpectation())
#     cvar_exp.compute_variance = lambda x: [0]  # to be fixed in PR #1373

    # initialize VQE using CVaR ###2 reps
    qaoa = QAOA(
        reps = 2,
        expectation=cvar_exp,
        optimizer=optimizer,
        quantum_instance=backend,
        callback=lambda i, params, obj, stddev: callback(i, params, obj, stddev, alpha),
    )

    # initialize optimization algorithm based on CVaR-VQE
    opt_alg = MinimumEigenOptimizer(qaoa)

    # solve problem
    results[alpha] = opt_alg.solve(qp)

    # print results
    print("alpha = {}:".format(alpha))
    print(results[alpha].prettyprint())
    print()  

alpha = 1.0:
objective function value: -0.25355531011501853
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, x_4=0.0, x_5=1.0, x_6=0.0, x_7=0.0, x_8=1.0, x_9=1.0
status: SUCCESS

alpha = 0.5:
objective function value: -0.25355531011501853
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, x_4=0.0, x_5=1.0, x_6=0.0, x_7=0.0, x_8=1.0, x_9=1.0
status: SUCCESS

alpha = 0.25:
objective function value: -0.25355531011501853
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, x_4=0.0, x_5=1.0, x_6=0.0, x_7=0.0, x_8=1.0, x_9=1.0
status: SUCCESS

alpha = 0.3:
objective function value: -0.25355531011501853
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, x_4=0.0, x_5=1.0, x_6=0.0, x_7=0.0, x_8=1.0, x_9=1.0
status: SUCCESS

alpha = 0.4:
objective function value: -0.25355531011501853
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, x_4=0.0, x_5=1.0, x_6=0.0, x_7=0.0, x_8=1.0, x_9=1.0
status: SUCCESS

alpha = 0.45:
objective function value: -0.25355531011501853
variable values: x_0=1.

### Conclusion:

1. In this case, for different alphas the optimal functional value remains the same. Hence, for this case we arrive at the       correct result with one repetition (reps = 1), thus saving us time and cost.



2. For this case with just 10 assets,we used qasm_simulator and COBYLA optimizer. Using different backends and classical optimizers would have not provided us with significantly better approximation for the optimal functional value than current backend-optimizer combination, thus we have opted against using other backend-optimizer combinations.